#1.라이브러리 불러오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import Series
import zipfile
from tqdm import tqdm
import numpy as np
from sklearn.linear_model import LinearRegression
import datetime

from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error


plt.rc("font", family="Malgun Gothic") # window
plt.rc("axes", unicode_minus=False) # x,y축 (-)부호 표시

#2.데이터 불러오기

In [ ]:
path_to_zip_file = '/content/drive/MyDrive/압축 파일' #압출 풀 데이터 경로
directory_to_extract_to = './task3' # 압축 푼 파일 저장 


with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [ ]:
train = pd.read_csv("/content/task3/train 데이터")

In [ ]:
test = pd.read_csv("/content/task3/test 데이터")

In [ ]:
result = pd.read_csv('/content/task3/result 데이터')

##1.train nan 값 처리

In [ ]:
train.isnull().sum()

In [29]:
train = train.fillna(method = 'ffill') #이전행으로

In [ ]:
train.isnull().sum()

#3.이상치 제거하기

In [31]:
#이상치 제거

def dr_outlier(df):
    quartile_1 = df.quantile(0.25)
    quartile_3 = df.quantile(0.75)
    IQR = quartile_3 - quartile_1

    condition = (df < (quartile_1 - 1.5 * IQR)) | (df > (quartile_3 + 1.5 * IQR))
    df[condition]=np.mean(df)
    
    return df


In [32]:
train['4단지배수지 공업 유출유량 적산차'] = dr_outlier(train['4단지배수지 공업 유출유량 적산차'])

<ipython-input-31-efd9d0f1aac1>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[condition]=np.mean(df)


#4.전처리

## 정규화

In [33]:
df_all = pd.concat([train, test])

In [34]:
df_all.index = df_all['datetime']
df_all = df_all.drop(['datetime'], axis = 1)

In [35]:
scaler = MinMaxScaler()
# scaler = RobustScaler()
df_y = scaler.fit_transform(df_all)

In [36]:
df_all['4단지배수지 공업 유출유량 적산차'] = df_y
df_all = df_all.reset_index(drop = False)

In [37]:
idx_train = len(train)
idx_test = len(test)

In [38]:
train = df_all.iloc[:idx_train]
test = df_all.iloc[idx_train:]

In [39]:
idx = []
for i in range(len(test)):
  idx.append(i)
test.index = idx

## 계절별로 전처리(3-5 : 0, 6-8 : 1, 9-11 : 2, 12-2 : 3)

In [40]:
train.columns = ['ds', 'y']

In [ ]:
train = train.astype({"ds":'str'})
train['year'] = [date.split('-')[0] for date in train['ds']]
train['month'] = [date.split('-')[1] for date in train['ds']]
train['day'] = [date.split('-')[2] for date in train['ds']]
train['day'] = [date.split(' ')[0] for date in train['day']]
train['hour'] = [date.split(' ')[-1] for date in train['ds']]
train['hour'] = [date.split(':')[0] for date in train['hour']]
train = train.drop(['ds'], axis = 1)
train[['year', 'month', 'day', 'hour']] = train[['year', 'month', 'day', 'hour']].astype(int)
train

In [44]:
dic_month2season = {1: 3, 
                    2: 3, 
                    3: 0, 
                    4: 0, 
                    5: 0, 
                    6: 1, 
                    7: 1, 
                    8: 1, 
                    9: 2, 
                    10: 2, 
                    11: 2, 
                    12: 3,
                   }

In [45]:
train["season"] = train["month"].map(dic_month2season)

#5.LinearRegression

##1.Train

In [46]:
X_train = train[["year", "month", "day", "hour", "season"]]
y_train = train["y"]

In [47]:
LR = LinearRegression()

In [48]:
LR.fit(X_train, y_train)

LinearRegression()

##2.Test

###1.예측할 날짜 추가(마지막날이 없어서)

In [49]:
lst_time = []
current_time = datetime.datetime.strptime(result["datetime"].loc[8424], "%Y-%m-%d %H:%M:%S")
lst_time.append(current_time)

for i in range(335):
    one_hour_later = current_time + datetime.timedelta(hours=1)
    current_time = one_hour_later
    lst_time.append(one_hour_later)

In [50]:
plus_time = pd.DataFrame(lst_time, columns=["datetime"])

In [51]:
test = pd.concat([test, plus_time], axis=0)
test["datetime"] = test["datetime"].astype(str)
test.reset_index(drop=True, inplace=True)

In [52]:
test.columns = ["ds", "y"]

###2.계절별로 전처리(3-5 : 0, 6-8 : 1, 9-11 : 2, 12-2 : 3)

In [53]:
test = test.astype({"ds":'str'})
test['year'] = [date.split('-')[0] for date in test['ds']]
test['month'] = [date.split('-')[1] for date in test['ds']]
test['day'] = [date.split('-')[2] for date in test['ds']]
test['day'] = [date.split(' ')[0] for date in test['day']]
test['hour'] = [date.split(' ')[-1] for date in test['ds']]
test['hour'] = [date.split(':')[0] for date in test['hour']]
test = test.drop(['ds'], axis = 1)
test[['year', 'month', 'day', 'hour']] = test[['year', 'month', 'day', 'hour']].astype(int)

In [54]:
test["season"] = test["month"].map(dic_month2season)

In [55]:
X_test = test[["year", "month", "day", "hour", "season"]]
y_test = test['y']

##3.predict

In [56]:
y_pred = LR.predict(X_test)

In [61]:
y_pred = scaler.inverse_transform(y_pred.reshape(-1,1))

##4.결과 집어넣기

In [66]:
def result_csv(df):
  df_y_pred = pd.DataFrame(data = df, columns=["y_pred"])

  lst_data = []

  for i in range(0, len(df)-335):
      data = df_y_pred["y_pred"].loc[i:i+335].values.tolist()
      lst_data.append(data) 

  df_submission = pd.DataFrame(data=lst_data, columns=result.columns[1:])

  file = pd.concat([result["datetime"], df_submission], axis=1)
  file.columns = result.columns

  evaluation2 = file.loc[:8424]

  mission_result=evaluation2[evaluation2.columns[:]]
  mission_result.to_csv('task3.csv',encoding='CP949',index=False)

  return mission_result

In [ ]:
result_csv(y_pred)